In [1]:
import requests,re,time,numpy as np,pickle,asyncio,concurrent.futures,datetime
# Make this notebook wiiiiiiiiide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

This parser works with just a few locations, constructing an ITA-style matrix of price vs start/end dates

In [2]:
# Setup relevant variables
ZIP = '60615'
AGENCYLIST = [('ET','E11509'), #oak park,
             #('ET','E11548'), #bolingbrook
             #('ET','E115CH'), #elmwood
             #('ET','E11516'), #skokie
             #('ET','E11553'), #Nn Lincolnwood 53
            ]
RADIUS = '50'

PICKUP = datetime.date(2017, 11, 29)
DROPOFF = PICKUP+datetime.timedelta(days=21)
FIXEDLENGTH = True
VARYSTART = 2
VARYEND = 2
TIME = '11:00 AM'

In [3]:
%run backend_withdb.ipynb

In [4]:
if not AGENCYLIST:
    agencylist = getagencylist((PICKUP,DROPOFF))
    AGENCYLIST = [(x['brand'],x['code']) for x in agencylist.values()]

In [ ]:
AGENCYLIST

In [5]:
populateagenciesdb((PICKUP,DROPOFF))

In [6]:
runparser(agencyids=AGENCYLIST, threads=1);

102 ND |  11/20/2017 12/11/2017 35 ('enterprise', 'E115CF', 'Sat Howard Orloff') 8.76
103 ND |  11/21/2017 12/12/2017 35 ('enterprise', 'E115CF', 'Sat Howard Orloff') 8.76
104 ND |  11/22/2017 12/13/2017 35 ('enterprise', 'E115CF', 'Sat Howard Orloff') 8.76
114 ND |  11/20/2017 12/11/2017 39 ('enterprise', 'E115FX', 'Nn Fox Ford') 9.58
115 ND |  11/21/2017 12/12/2017 39 ('enterprise', 'E115FX', 'Nn Fox Ford') 9.58
116 ND |  11/22/2017 12/13/2017 39 ('enterprise', 'E115FX', 'Nn Fox Ford') 9.58
119 ND |  11/22/2017 12/13/2017 40 ('enterprise', 'E11510', 'Nn Lincoln Park') 9.61
167 ND |  11/22/2017 12/13/2017 56 ('enterprise', 'E11558', 'Nn Uptown') 12.59
279 ND |  11/20/2017 12/11/2017 94 ('enterprise', 'E115P1', 'Packer Branch Elmhurst') 19.34
280 ND |  11/21/2017 12/12/2017 94 ('enterprise', 'E115P1', 'Packer Branch Elmhurst') 19.34
281 ND |  11/22/2017 12/13/2017 94 ('enterprise', 'E115P1', 'Packer Branch Elmhurst') 19.34
321 ND |  11/20/2017 12/11/2017 108 ('enterprise', 'E115PD', 'P

In [ ]:
res = session.query(Agency).filter(Agency.ag_id == 'E11509').first()
print(res)
res.prices

In [ ]:
session.query(Price.car_econ, Price.pickup, Agency.ag_dist, Agency.ag_name).join(Agency).filter(Price.car_econ.isnot(None)).order_by(Price.car_comp).count()

In [7]:
res = session.query(Price.car_int, Price.pickup, Agency.ag_dist, Agency.ag_name).join(Agency).filter(Price.car_econ.isnot(None)).order_by(Price.car_int).all()

In [8]:
import pprint
pprint.pprint(res)

[(438.0, '11/20/2017', 44.16, 'Gurnee Il Sears'),
 (438.0, '11/21/2017', 44.16, 'Gurnee Il Sears'),
 (442.17, '11/20/2017', 29.21, 'Highland Park'),
 (442.17, '11/21/2017', 29.21, 'Highland Park'),
 (442.17, '11/20/2017', 37.18, 'Lake Zurich Il'),
 (442.17, '11/21/2017', 37.18, 'Lake Zurich Il'),
 (442.17, '11/20/2017', 37.57, 'Libertyville'),
 (442.17, '11/21/2017', 37.57, 'Libertyville'),
 (442.17, '11/20/2017', 39.56, 'Libertyville Il'),
 (442.17, '11/21/2017', 39.56, 'Libertyville Il'),
 (442.17, '11/20/2017', 46.33, 'Crystal Lake Il'),
 (442.17, '11/21/2017', 46.33, 'Crystal Lake Il'),
 (462.89, '11/20/2017', 44.16, 'Sears Auto Center - Gurnee Mills Mall'),
 (462.89, '11/21/2017', 44.16, 'Sears Auto Center - Gurnee Mills Mall'),
 (467.2, '11/20/2017', 25.02, 'Northbrook'),
 (467.2, '11/21/2017', 25.02, 'Northbrook'),
 (467.29999999999995, '11/20/2017', 29.21, 'Highland Park Ford'),
 (467.29999999999995, '11/21/2017', 29.21, 'Highland Park Ford'),
 (467.29999999999995,
  '11/21/201

In [ ]:
import pprint
pprint.pprint(res)

In [ ]:
agencylist = getagencylist((PICKUP,DROPOFF))

In [ ]:
print(agencylist)

In [ ]:
results = runparser(agencyids = AGENCYIDS, threads=5)

In [ ]:
print(results)

In [ ]:
car1 = 'Economy Car'
car2 = 'Compact Car'
car3 = 'Intermediate Car'
car4 = 'Intermediate SUV'
desiredcar = car2
#desiredsort = 'time'
desiredsort = desiredcar
desiredbrand = 'enterprise'

# also, can optimize into some single query...buuuut it works as is
rvals = np.array([x for x in results.values() if x is not None and desiredcar in x and desiredsort in x])
orderpreferred = np.array([x[desiredsort] for x in rvals]).argsort()
orderprice = np.array([x[desiredcar] for x in rvals]).argsort()
order = [x for x in orderpreferred if x in orderprice[:60]]

In [ ]:
orderedresults = rvals[order]
print(desiredcar,car2,car3)
for k in orderedresults:
    #print(k['num'])
    if 'time' in k:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),k['time']//60,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))
    else:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),0,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))